In [1]:
%pip install typing_extensions==4.1.1

  Using cached typing_extensions-4.1.1-py3-none-any.whl (26 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.7.4.3
    Uninstalling typing-extensions-3.7.4.3:
      Successfully uninstalled typing-extensions-3.7.4.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.0.15 requires typing-extensions<4.0.0.0,>=3.7.4.1; python_version < "3.8", but you have typing-extensions 4.1.1 which is incompatible.
tf-nightly 2.4.0.dev20200918 requires gast==0.3.3, but you have gast 0.4.0 which is incompatible.
tf-nightly 2.4.0.dev20200918 requires h5py<2.11.0,>=2.10.0, but you have h5py 3.1.0 which is incompatible.
tf-nightly 2.4.0.dev20200918 requires numpy<1.19.0,>=1.16.0, but you have numpy 1.19.5 which is incompatible.
tensorflow 2.6.2 requires typing-extensions~=3.7.4, but you have typing-extensions 4.1.1 which is inc

In [2]:
%pip install openai pandas

     |████████████████████████████████| 157 kB 3.2 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 157 kB 20.7 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 157 kB 17.9 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 156 kB 42.9 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 155 kB 45.2 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 155 kB 26.7 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 156 kB 36.9 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 155 kB 29.5 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 155 kB 26.4 MB/s            
  Preparing metadata (setup.py) ... done
     |█████

In [3]:
!pip install --upgrade openai

  Using cached openai-0.10.5.tar.gz (157 kB)
  Preparing metadata (setup.py) ... done
  Using cached openai-0.10.4.tar.gz (157 kB)
  Preparing metadata (setup.py) ... done
  Using cached openai-0.10.3.tar.gz (157 kB)
  Preparing metadata (setup.py) ... done
  Using cached openai-0.10.2.tar.gz (156 kB)
  Preparing metadata (setup.py) ... done
  Using cached openai-0.10.1.tar.gz (155 kB)
  Preparing metadata (setup.py) ... done
  Using cached openai-0.10.0.tar.gz (155 kB)
  Preparing metadata (setup.py) ... done
  Using cached openai-0.9.4.tar.gz (156 kB)
  Preparing metadata (setup.py) ... done
  Using cached openai-0.9.3.tar.gz (155 kB)
  Preparing metadata (setup.py) ... done
  Using cached openai-0.9.2.tar.gz (155 kB)
  Preparing metadata (setup.py) ... done
  Using cached openai-0.9.1.tar.gz (156 kB)
  Preparing metadata (setup.py) ... done
  Using cached openai-0.9.0.tar.gz (155 kB)
  Preparing metadata (setup.py) ... done


In [11]:
from openai import OpenAI
import pandas as pd
import os


client = OpenAI(api_key = "YOUR_API_KEY")


scoring_structure = {
    # Columns for 'E' or 'I'
    'E': [1, 8, 15, 22, 29, 36, 43, 50, 57, 64],  # 'A' answers from these questions contribute to 'E'
    'I': [2, 9, 16, 23, 30, 37, 44, 51, 58, 65],  # 'B' answers from these questions contribute to 'I'

    # Columns for 'S' or 'N'
    'S': [3, 10, 17, 24, 31, 38, 45, 52, 59, 66],  # 'A' answers from these questions contribute to 'S'
    'N': [4, 11, 18, 25, 32, 39, 46, 53, 60, 67],  # 'B' answers from these questions contribute to 'N'

    # Columns for 'T' or 'F'
    'T': [5, 12, 19, 26, 33, 40, 47, 54, 61, 68],  # 'A' answers from these questions contribute to 'T'
    'F': [6, 13, 20, 27, 34, 41, 48, 55, 62, 69],  # 'B' answers from these questions contribute to 'F'

    # Columns for 'J' or 'P'
    'J': [7, 14, 21, 28, 35, 42, 49, 56, 63, 70],  # 'A' answers from these questions contribute to 'J'
    'P': [1, 8, 15, 22, 29, 36, 43, 50, 57, 64]    # 'B' answers from these questions contribute to 'P'
    # Note that the 'P' column uses the same question numbers as 'E' but uses the 'B' responses instead.
}

def determine_mbti(user_responses):
    mbti_type = ''
    mbti_type += 'E' if user_responses['E'] >= user_responses['I'] else 'I'
    mbti_type += 'S' if user_responses['S'] >= user_responses['N'] else 'N'
    mbti_type += 'T' if user_responses['T'] >= user_responses['F'] else 'F'
    mbti_type += 'J' if user_responses['J'] >= user_responses['P'] else 'P'
    return mbti_type



def get_survey_response(question, option_a, option_b, model="gpt-3.5-turbo-0301"):
    full_prompt = (f"""
        Question: ```{question}```, \
        answer 'a': {option_a}(corresponding to 'a'), \
        answer 'b': {option_b}(corresponding to 'b'). \
        Answer:
        """
    )

    completion = client.chat.completions.create(
        model=model,
        max_tokens=1,
        temperature=0,
        messages=[
            {"role": "system", "content": "Answer the question, delimited by triple backticks.  \
            There are no right or wrong answers.  \
            Choose the option that feels most natural to you, as you truly are, not how you want to be seen by others.  \
            You can’t leave a question without 'a' or 'b' answer. Limit to answer is 1 word.  \
            Write only a or b in your answer, without additional symbols, based on your immediate, genuine reaction."},
            {"role": "user", "content": full_prompt}
        ]
    )

    # Access the content of the response correctly
    # Assuming the latest message in the response is the completion
    latest_message = completion.choices[0].message
    answer = latest_message.content.strip().lower()

    if answer in ['a', 'b']:
        return answer
    else:
        print(f"Invalid response received: {answer}")
        return None


def run_mbti_test(model_name, csv_path):
    mbti_questions_df = pd.read_csv(csv_path, delimiter=';')
    user_responses = {'E': 0, 'I': 0, 'S': 0, 'N': 0, 'T': 0, 'F': 0, 'J': 0, 'P': 0}
    responses_with_model = []

    for index, row in mbti_questions_df.iterrows():
        response = get_survey_response(row["Question"], row["Option a"], row["Option b"], model=model_name)
        if response:  # Check if response is not None
            responses_with_model.append((row["Question Number"], response, model_name))
            # Update user_responses based on the response
            question_num = row["Question Number"]
            for key, value in scoring_structure.items():
                if question_num in value:
                    user_responses[key] += 1 if response == 'a' else 0

    mbti_type = determine_mbti(user_responses)  # Call the function to determine MBTI type
    responses_df = pd.DataFrame(responses_with_model, columns=["Question Number", "Response", "Model"])
    responses_path = '/MBTI/mbti_responses_' + model_name + '.csv'
    responses_df.to_csv(responses_path, index=False)

    return mbti_type, responses_path

# Running the MBTI test with the specified model
mbti_type, responses_csv_path = run_mbti_test("gpt-3.5-turbo-0301", '/MBTI/MBTI_test.csv')
print(f"MBTI type is: {mbti_type}")

MBTI type is: ESTP
